In [ ]:
import tg_plots
import graph_io
import utils
import sampling
import generators
import temporal_graph

import os
import math
import collections
import statistics

import powerlaw
import scipy.optimize
import numpy as np
import pandas as pd
import pytglib as tgl
import seaborn as sns
import networkit as nk
import matplotlib.pyplot as plt

sns.set_theme()

temporal clustering with y-axis log scale

In [ ]:
from matplotlib import ticker as mticker

data = pd.read_csv("data/graphs_stats/random_from_real_world.csv").query("temp_Clustering > 0")

data["temp_Clustering"] = data["temp_Clustering"].map(np.log10)

sns.catplot(data, x="Kategorie", y="temp_Clustering", kind="violin", height=5, aspect=2)

ax = plt.gca()
ax.yaxis.set_major_formatter(mticker.StrMethodFormatter("$10^{{{x:.0f}}}$"))
ymin, ymax = ax.get_ylim()
tick_range = np.arange(np.floor(ymin), 1)
ax.yaxis.set_ticks(tick_range)
ax.yaxis.set_ticks([np.log10(x) for p in tick_range for x in np.linspace(10 **p, 10 **(p + 1), 10)], minor=True)

plt.xlabel("Graph Kategorie")
plt.ylabel("temporales Clustering")
plt.savefig("data/graphics/temp_clustering.pdf", bbox_inches="tight")
plt.show()

Plot a single graph

In [ ]:
path = "data/temporal_graphs/communication/Enron.edges"

tg_plots.edge_cardinality_distribution(path)
plt.xlabel("Kanten-Kardinalität")
plt.ylabel("Anzahl")
plt.show()

tg_plots.timestamp_distribution(path, artifacts=0.45, scatter=False)
plt.xlabel("Zeitstempel")
plt.ylabel("Anzahl")
plt.show()

tg_plots.degree_distribution(path)
plt.xlabel("Grad")
plt.ylabel("Anzahl")
plt.show()

Plot all graphs of a category

In [ ]:
tg_plots.plot_graphs(tg_plots.TemporalGraphCategories.SOCIAL_MEDIA_GRAPHS, tg_plots.edge_cardinality_distribution, x_label="Kardinalität", y_label="Anzahl", ncols=4)

In [ ]:
tg_plots.plot_graphs(tg_plots.TemporalGraphCategories.SOCIAL_MEDIA_GRAPHS, tg_plots.timestamp_distribution, x_label="Zeitstempel", y_label="Anzahl", ncols=2, plot_width=10, artifacts=0.45, scatter=False)

In [ ]:
tg_plots.plot_graphs(tg_plots.TemporalGraphCategories.SOCIAL_MEDIA_GRAPHS, tg_plots.degree_distribution, x_label="Grad", y_label="Anzahl", ncols=4)

Calculate stats

In [ ]:
tg_plots.stats(tg_plots.iter_temporal_graphs(tg_plots.TemporalGraphCategories.values(), return_type=True), directed=lambda x: False)

In [ ]:
folder = "edge_exponent_test"
folders = map(lambda x: os.path.join(folder, x), os.listdir(f"data/temporal_graphs/{folder}"))
tg_plots.stats(tg_plots.iter_temporal_graphs(folders, return_type=True), directed=lambda x: False)

Plot stats

In [ ]:
outside_legend=False
aspect = 2

# temporal metrics
tg_plots.avg_reachability(outside_legend)
tg_plots.avg_reachability_ratio(outside_legend)
tg_plots.temporal_diameter(outside_legend)
tg_plots.temporal_correlation_coefficient(aspect)
tg_plots.temporal_clustering(aspect)
tg_plots.edge_cardinality_exponent(aspect)

# static metrics
tg_plots.avg_degree(outside_legend)
tg_plots.assortativity(aspect)
tg_plots.static_clustering(aspect)
tg_plots.degree_exponent(aspect)

In [ ]:
tg_plots.connected_components()

In [ ]:
data = pd.read_csv(tg_plots.GRAPH_STATS_PATH).query("Erreichbarkeit > 0")

sns.scatterplot(data, x="Knoten", y="Erreichbarkeit", hue="Zufallsgraph", style="Kategorie")
# plt.plot(*zip(*[(x, x) for x in range(1, max(data["Knoten"]))]), label="n")
plt.loglog()
plt.xlabel("Knoten")
plt.ylabel("durchschnittliche Erreichbarkeit")
tg_plots.show_legend(True)
# plt.savefig("data/graphics/reachability.pdf", bbox_inches="tight")
plt.show()

Erdős–Rényi Graphs: connected component and giant component

In [ ]:
for path in tg_plots.iter_temporal_graphs(tg_plots.TemporalGraphCategories.values()):
    graph = graph_io.read_temporal_graph(path, directed=False, static=True)
    n = graph.numberOfNodes()
    m = graph.numberOfEdges()
    p = (2 * m) / (n * (n - 1))
    print(utils.file_name(path).replace("_", " "), n, m, "Ja" if n * p > 1 else "Nein", "Ja" if n * p > math.log(n) else "Nein", sep=" & ", end=" \\\\\n")

In [ ]:
data = []

for category in tg_plots.TemporalGraphCategories.values():
    giant_component = 0
    connected = 0
    for path in tg_plots.iter_temporal_graphs([category]):
        graph = graph_io.read_temporal_graph(path, directed=False, static=True)
        n = graph.numberOfNodes()
        m = graph.numberOfEdges()
        p = (2 * m) / (n * (n - 1))

        giant_component += n * p > 1
        connected += p > math.log(n) / n

    print(category, giant_component, connected)

Havel-Hakimi connected components

In [ ]:
folder_path = "data/temporal_graphs/random_from_real_world/havel-hakimi/"

components_data = []

for path in sorted(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, path)
    # directed = tg_plots.is_directed(file_path.replace("-random", ""))
    directed = False

    graph = graph_io.read_temporal_graph(file_path, directed=directed, static=True)
    number_of_components = None
    largest_component = None

    if graph.numberOfNodes() > 1_000_000:
        continue

    if directed:
        scc = nk.components.StronglyConnectedComponents(graph).run()
        partition = scc.getPartition()
        indexes = sorted(set(partition.getVector()))
        largest_component = max(len(partition.getMembers(cmp)) for cmp in indexes)
        number_of_components = scc.numberOfComponents()
    else:
        cc = nk.components.ConnectedComponents(graph).run()
        largest_component = cc.extractLargestConnectedComponent(graph, True).numberOfNodes()
        number_of_components = cc.numberOfComponents()

    print(path, directed, number_of_components, graph.numberOfNodes(), largest_component, round(largest_component / graph.numberOfNodes(), 2))

    components_data.append([graph.numberOfNodes(), number_of_components, largest_component])

components_data = pd.DataFrame(components_data, columns=["Nodes", "Components", "Largest-Component"])

In [ ]:
components_data["Ratio"] = components_data["Largest-Component"] / components_data["Nodes"]

sns.scatterplot(data=components_data, x="Nodes", y="Ratio")
plt.xscale("log")
plt.ylim(0, 1)
plt.xlabel("Knoten")
plt.ylabel("Anteil der größten Zusammenhangskomponente")
plt.savefig("data/graphics/random/largest-component.pdf", bbox_inches="tight")
plt.show()

In [ ]:
print(components_data["Ratio"].mean())

In [ ]:
sns.scatterplot(data=components_data, x="Nodes", y="Components")
plt.loglog()
plt.xlabel("Knoten")
plt.ylabel("Zusammenhangskomponenten")
plt.savefig("data/graphics/random/number_of_components.pdf", bbox_inches="tight")
plt.show()

In [ ]:
def func(x, a):
    return a * x

components_data.sort_values(by=["Nodes"], inplace=True)

popt, _ = scipy.optimize.curve_fit(func, components_data["Nodes"], components_data["Components"])
print(popt)

sns.scatterplot(components_data, x="Nodes", y="Components")
plt.plot(components_data["Nodes"], func(components_data["Nodes"], *popt), label="fit")
plt.loglog()

Chung-Lu isolated nodes

In [ ]:
real_world_stats = pd.read_csv("data/graphs_stats/real_world.csv")
random_stats = pd.read_csv("data/graphs_stats/random_from_real_world.csv").query("Kategorie == 'Chung-Lu'")

def get_number_of_nodes(dataset):
    dataset = dataset.replace("-random", "")
    return real_world_stats.query(f"Datensatz == @dataset")["Knoten"].iloc[0]


random_stats["Knoten_Original"] = random_stats["Datensatz"].map(get_number_of_nodes)
random_stats["Isoliert"] = random_stats["Knoten_Original"] - random_stats["Knoten"]
random_stats["Anteil"] = random_stats["Isoliert"] / random_stats["Knoten_Original"]

print(random_stats["Anteil"].mean())

sns.scatterplot(data=random_stats, x="Knoten_Original", y="Anteil")
plt.xscale("log")
plt.ylim(0, 1)
plt.xlabel("Knoten")
plt.ylabel("Anteil der isolierten Knoten")
plt.savefig("data/graphics/random/isolated.pdf", bbox_inches="tight")
plt.show()

Generate random temporal graphs from real world temporal graphs

In [ ]:
folder_path = os.path.join(tg_plots.TEMPORAL_GRAPHS_PATH, "random_from_real_world", "havel-hakimi")

for network_type, path in tg_plots.iter_temporal_graphs(tg_plots.TemporalGraphCategories.values(), return_type=True):
    # if "MIT_Reality_Mining" in path: continue

    edges = generators.havel_hakimi_tg_from_graph(path, tg_plots.is_directed(path))
    random_graph_path = os.path.join(folder_path f"{utils.file_name(path)}-random.edges")
    graph_io.write_edge_list(random_graph_path, edges)

Time interval experiments

In [ ]:
for days in [1, 10, 20]:
    # folder_path = f"data/temporal_graphs/time_interval_test/chung-lu_waves_{days}"
    folder_path = f"data/temporal_graphs/time_interval_test/hyperbolic_waves_{days}"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for i, n in enumerate(utils.node_count_iterator(100, 20_000)):
        timestamps_per_day = 24
        timestamps = days * timestamps_per_day
        timestamp_weights = sampling.generate_burst_weights(days, timestamps_per_day // 2)

        cardinality_distribution = sampling.Zipf(2, timestamps)

        G = nk.generators.HyperbolicGenerator(n, 10, 2.5).generate()

        # degree_seq = nk.generators.PowerlawDegreeSequence(1, n - 1, -2.5).run().getDegreeSequence(n)
        # G = nk.generators.ChungLuGenerator(degree_seq).generate()

        edges = generators.temporal_graph_generator(G, cardinality_distribution, timestamp_weights)
        path = os.path.join(folder_path, f"{i:02d}.edges")
        graph_io.write_edge_list(path, edges)

In [ ]:
for timestamps in [1, 2, 5, 10, 50, 100, 200]:
    # folder_path = f"data/temporal_graphs/time_interval_test/chung-lu_uniform_{timestamps}"
    folder_path = f"data/temporal_graphs/time_interval_test/hyperbolic_uniform_{timestamps}"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for i, n in enumerate(utils.node_count_iterator(100, 20_000)):
        timestamp_weights = [1] * timestamps
        
        cardinality_distribution = sampling.Zipf(2, timestamps)

        G = nk.generators.HyperbolicGenerator(n, 10, 2.5).generate()

        # degree_seq = nk.generators.PowerlawDegreeSequence(1, n - 1, -2.5).run().getDegreeSequence(n)
        # G = nk.generators.ChungLuGenerator(degree_seq).generate()

        edges = generators.temporal_graph_generator(G, cardinality_distribution, timestamp_weights)
        path = os.path.join(folder_path, f"{i:02d}.edges")
        graph_io.write_edge_list(path, edges)

Avg. degree experiments

In [ ]:
for avg_degree in [2, 4, 7, 10, 15]:
    folder_path = f"data/temporal_graphs/avg_degree_test/avg-degree-{avg_degree}"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for i, n in enumerate(utils.node_count_iterator(100, 20_000)):
        timestamps = 100_000
        timestamp_weights = [1] * timestamps
        
        cardinality_distribution = sampling.Zipf(2, timestamps)

        G = nk.generators.HyperbolicGenerator(n, avg_degree, 2.5).generate()

        edges = generators.temporal_graph_generator(G, cardinality_distribution, timestamp_weights)
        path = os.path.join(folder_path, f"{i:02d}.edges")
        graph_io.write_edge_list(path, edges)

Edge cardinalities exponent experiments

In [ ]:
for min_c, exp in [(1, 1.5), (30, 2.0), (83, 2.5)]:
    folder_path = f"data/temporal_graphs/edge_exponent_test/hyperbolic_{exp}"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for i, n in enumerate(utils.node_count_iterator(100, 20_000)):
        timestamps = range(100_000)
        timestamp_weights = [1] * len(timestamps)

        G = nk.generators.HyperbolicGenerator(n, 10, 2.5).generate()

        sampler = sampling.Hybrid(timestamps, timestamp_weights)
        cardinalities = nk.generators.PowerlawDegreeSequence(min_c, len(timestamps), -exp).run().getDegreeSequence(G.numberOfEdges())

        edges = []
        for (u, v), k in zip(G.iterEdges(), cardinalities):
            for t in sampler.sample(k):
                edges.append((u, v, t))

        path = os.path.join(folder_path, f"{i:02d}.edges")
        graph_io.write_edge_list(path, edges)

In [ ]:
data = pd.read_csv(tg_plots.GRAPH_STATS_PATH)

sns.scatterplot(data, x="Knoten", y="Kanten", hue="Kategorie")
plt.loglog()
plt.savefig("data/graphics/edges.pdf")
plt.show()

Granularity experiment

In [ ]:
graph_path = "data/temporal_graphs/communication/Enron.edges"
folder_path = "data/temporal_graphs/granularity_test"

for i in range(10):
    edges = graph_io.read_edge_list(graph_path)
    edges = list({(u,v,t // (10**i), tt) for (u,v,t,tt) in edges})
    graph_io.write_edge_list(os.path.join(folder_path, f"{i}.edges"), edges)
    print(len(edges))

In [ ]:
folder_path = "data/temporal_graphs/granularity_test"

for path in sorted(os.listdir(folder_path)):
    tg = temporal_graph.TemporalGraph(os.path.join(folder_path, path), True)
    timestamps = tg.timestamps()
    print(path, min(timestamps), max(timestamps), max(timestamps) - min(timestamps) + 1, len(set(timestamps)))

In [ ]:
tg_plots.stats(tg_plots.iter_temporal_graphs(["granularity_test"], return_type=True), directed=lambda x: True)

In [ ]:
data = pd.read_csv("data/graphs_stats/granularity_test.csv")
print(data.to_markdown(index=False))

In [ ]:
tg_plots.plot_graphs("granularity_test", tg_plots.edge_cardinality_distribution)

Power Law

In [ ]:
generator = nk.generators.PowerlawDegreeSequence(83, 100_000, -2.5).run()
print(generator.getExpectedAverageDegree())
print(sum(generator.getDegreeSequence(100_000)) / 100_000)

In [ ]:
def harmonic_number(n, gamma=1):
    return sum(1 / (i**gamma) for i in range(1, n + 1))


def expected_value(x_min, x_max, gamma):
    return (harmonic_number(x_max, gamma - 1) - harmonic_number(x_min - 1, gamma - 1)) / (harmonic_number(x_max, gamma) - harmonic_number(x_min - 1, gamma))

print(expected_value(83, 100_000, 2.5))

Plot temporal diameter of random graphs

In [ ]:
data_real_world = pd.read_csv("data/graphs_stats/real_world.csv")
data_random = pd.read_csv("data/graphs_stats/random_from_real_world.csv")

sns.scatterplot(data_random, x="Knoten", y="temp_Durchmesser", hue="Kategorie")
sns.scatterplot(data_real_world, x="Knoten", y="temp_Durchmesser", hue="Kategorie", marker="+", palette=['pink','orange','dodgerblue','red'])
plt.plot(*zip(*[(x, math.log2(x)) for x in range(1, max(data_real_world["Knoten"]))]), label="log2(n)")
plt.xlabel("Knoten")
plt.ylabel("temporaler Durchmesser")
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.loglog()
plt.xlim(10, 10**6)
plt.show()

Fit temporal diameter

In [ ]:
"""
def func(x, base, a, c):
    return a* (np.log(x) / np.log(base)) + c
"""
def func(x, base):
    return np.log(x) / np.log(base)

    
data = pd.read_csv("data/graphs_stats/real_world.csv").query("temp_Durchmesser > 0")
data.sort_values(by=["Knoten"], inplace=True)

popt, _ = scipy.optimize.curve_fit(func, data["Knoten"], data["temp_Durchmesser"], bounds=[[1], [20]])
print(popt)

sns.scatterplot(data, x="Knoten", y="temp_Durchmesser")
plt.plot(data["Knoten"], func(data["Knoten"], *popt), label="fit")
plt.plot(*zip(*[(x, math.log(x, 2)) for x in range(1, max(data["Knoten"]))]), label="log2(n)")
plt.loglog()
plt.legend()
plt.show()

In [ ]:
def func(x, k):
    return np.power(x, k)

data = pd.read_csv("data/graphs_stats/real_world.csv").query("Erreichbarkeit > 0")
data.sort_values(by=["Knoten"], inplace=True)

popt, _ = scipy.optimize.curve_fit(func, data["Knoten"], data["Erreichbarkeit"])
print(popt)

sns.scatterplot(data, x="Knoten", y="Erreichbarkeit")
plt.plot(data["Knoten"], func(data["Knoten"], *popt), label="fit")
plt.loglog()
plt.legend()
plt.show()

In [ ]:
data_real_world = pd.read_csv("data/graphs_stats/real_world.csv").query("temp_Durchmesser > 0")
data_random = pd.read_csv("data/graphs_stats/random_from_real_world.csv").query("temp_Durchmesser > 0")

def get_real_world_diameter(dataset):
    dataset = dataset.replace("-random", "")
    return data_real_world.query(f"Datensatz == @dataset")["temp_Durchmesser"].iloc[0]


data_random["beobachtet_Durchmesser"] = data_random["Datensatz"].map(get_real_world_diameter)
data_random["Verhaeltnis"] = data_random["temp_Durchmesser"] / data_random["beobachtet_Durchmesser"]

sns.catplot(data_random, x="Kategorie", y="Verhaeltnis", kind="violin", height=5, aspect=2)
plt.yticks([0, 1, 2, 3, 6, 9, 12, 15])
plt.ylim(0, 15)
plt.xlabel("Graph Kategorie")
plt.ylabel("Verhältnis")
plt.savefig("data/graphics/diameter_comparison.pdf", bbox_inches="tight")
plt.show()

Real world graphs nodes

In [ ]:
data = pd.read_csv("data/graphs_stats/real_world.csv")

for category in np.unique(data["Kategorie"]):
    data_category = data.query("Kategorie == @category")
    print(category, data_category["Knoten"].min(), data_category["Knoten"].max(), data_category["Knoten"].median())